# MiniMax TicTacToe

In this notebook we will explore the famous MiniMax Algorithm. To do so we will use the simple game of TicTacToe:

![alt text](https://upload.wikimedia.org/wikipedia/commons/d/da/TicTacToe-152374698XOp.gif "Demo GIF")

The rules of that game are simple, align 3 of your pieces horizontally, vertically or diagonaly to win. In this notebook, we will approach the MiniMax algorithm from a naive standpoint by exploring simple algorithms. The first one will be a one step lookahead. Yet let us firstly look at the game environment that is in file "TicTacToe.py". The default strategy of the agent is random decision making:

In [1]:
import TicTacToe
game = TicTacToe.TicTacToe()
game.play()
game.percentage()

-| -| -| 
-| -| -| 
-| -| -| 

-| -| -| 
-| -| -| 
-| X| -| 

-| -| -| 
-| -| O| 
-| X| -| 

-| -| -| 
-| -| O| 
X| X| -| 

-| -| -| 
-| -| O| 
X| X| O| 

-| -| X| 
-| -| O| 
X| X| O| 

-| O| X| 
-| -| O| 
X| X| O| 

X| O| X| 
-| -| O| 
X| X| O| 

X| O| X| 
-| O| O| 
X| X| O| 

X| O| X| 
X| O| O| 
X| X| O| 

Player 1 won!
Player 1: random vs. Player 2: random
Player 1 W: 40, L: 46, T: 14 

It makes sense that two random agents playing against each other would almost tie. Now let us explore the **One Step Lookahead** algorithm. This algorithm simply looks at the board and finds out if there exist a move that would win the game and plays it accordingly. 

```python
    def agent_next_winning_move(self):
        # Agent that plays the next move that wins the board
        board_copy = self.game_board.copy()
        moves = self.get_valid_moves()
        next_move = None
        for move in moves:
            x, y = int(move / 3), int(move % 3)
            board_copy[x][y] = self.player_turn
            for i in range(0, 3):
                # Vertical winner
                if board_copy[0][i] != '-':
                    if self.game_board[0][i] == board_copy[1][i] == board_copy[2][i]:
                        next_move = move
                        break
                # Horizontal winner
                if board_copy[i][0] != '-':
                    if board_copy[i][0] == board_copy[i][1] == board_copy[i][2]:
                        next_move = move
                        break
            # Diagonal winner
            if board_copy[1][1] != '-':
                if board_copy[0][0] == board_copy[1][1] == board_copy[2][2]:
                    next_move = move
                    break
                elif board_copy[2][0] == board_copy[1][1] == board_copy[0][2]:
                    next_move = move
                    break
            # Tie
            for i in range(3):
                for j in range(3):
                    # If there is at least one tile not filed
                    if board_copy[i][j] == '-':
                        next_move = None

            next_move = None
        #
        if next_move is not None:
            return int(next_move / 3), int(next_move % 3)
        else:
            xy = random.choice(moves)
            return int(xy / 3), int(xy % 3)

```

In [4]:
from TicTacToe import TicTacToe
game = TicTacToe(player1="next_winning", player2="random")
game.play()
game.percentage()

| -| -| -| 
| -| -| -| 
| -| -| -| 

| -| -| -| 
| -| -| -| 
| X| -| -| 

| O| -| -| 
| -| -| -| 
| X| -| -| 

| O| -| -| 
| -| X| -| 
| X| -| -| 

| O| -| -| 
| -| X| -| 
| X| -| O| 

| O| -| X| 
| -| X| -| 
| X| -| O| 

Player 1 won!
Player 1: next_winning vs. Player 2: random
Player 1 W: 64, L: 29, T: 7 

The results are already promissing.

Now the State Space of TicTacToe (the 9 tiles) is quite limited so the One Step Lookahead is quite easy to develop. For more complex games like chess, developping a One Step Lookahead algorithm is more challenging. Indeed our TicTacToe script picks a random position when there is no winning move available. Yet for chess the approach to take would be to build a heuristic function. Even though I have not built one for this project, we can look at the purpose of such a function. As a player, there are position that are more advantageous; the heuristic function simply translates such preferences into a fitness score and the move with the highest score will be selected. In chess we could translate the capture of each piece with a reward which will direct the algorithm to prioritize important pieces over pawns.

Given the processing power of computers, why not elaborate the One Step Lookahead to a N Step Lookahead. Instead of simulating a single move and looking at the outcome, we can simulate all possible sets of N moves and observe the outcomes. This method is called the Monte Carlo Simulation.
```python
    def n_step_rec(self, turn, depth):
        # Recursive function that explores different combinations of moves
        self.result = self.winner()

        # If the Game is over output the outcome
        if self.result != None:
            if self.result == 'X':
                return 1
            elif self.result == 'O':
                return -1
            elif self.result == '-':
                return 0
        # If the number of exploration reaches a maximum
        if depth == 0:
            return 0
        valid_moves = self.get_valid_moves()
        # Simulate a move
        if turn == 'X':
            value = 0
            for move in valid_moves:
                x, y = int(move / 3), int(move % 3)
                self.game_board[x][y] = turn
                value += self.n_step_rec('O', depth - 1)
                self.game_board[x][y] = '-'
            return value
        else:
            value = 0
            for move in valid_moves:
                x, y = int(move / 3), int(move % 3)
                self.game_board[x][y] = turn
                value += self.n_step_rec('X', depth - 1)
                self.game_board[x][y] = '-'

            return value
```
The code is not much harder than previously, it simply extends the search deeper. This is a strategy that humans do subconsciously when playing board games. In chess professionals try to plan several steps ahead to find the optimal move.

In [1]:
from TicTacToe import TicTacToe
game = TicTacToe(player1="n_step", player2="random")
game.play()
game.percentage()

| -| -| -| 
| -| -| -| 
| -| -| -| 

| -| -| -| 
| -| X| -| 
| -| -| -| 

| -| -| -| 
| O| X| -| 
| -| -| -| 

| X| -| -| 
| O| X| -| 
| -| -| -| 

| X| -| O| 
| O| X| -| 
| -| -| -| 

| X| -| O| 
| O| X| -| 
| -| X| -| 

| X| -| O| 
| O| X| -| 
| -| X| O| 

| X| X| O| 
| O| X| -| 
| -| X| O| 

Player 1 won!
Player 1: n_step vs. Player 2: random
Player 1 W: 79, L: 5, T: 16 

The results are getting even better yet. The Monte Carlo Tree Search simulates blindly all game states. Yet The ***Minimax*** algorithm will put the assumption that the opponent attempts to play optimally. Indeed, the algorithm will find the move that progressively optimize its winning chances against an agent that attempts to minimize it. The approach taken is to simulate games, record the outcomes and find the path where at each move, we pick the best move for us and the opponent picks the worst one for us.
```python
    # Minimax implementation
    def minimax(self, turn, depth):
        # Recursive function that explores different combinations of moves
        self.result = self.winner()

        # If the Game is over output the outcome
        if self.result != None:
            if self.result == 'X':
                return 1
            elif self.result == 'O':
                return -1
            elif self.result == '-':
                return 0
        # If the number of exploration reaches a maximum
        if depth == 0:
            return 0

        valid_moves = self.get_valid_moves()
        if turn == 'X':
            value = -np.Inf
            for move in valid_moves:
                x, y = int(move / 3), int(move % 3)
                self.game_board[x][y] = turn
                value = max(value, self.n_step_rec('O', depth - 1))
                self.game_board[x][y] = '-'
            return value
        else:
            value = np.Inf
            for move in valid_moves:
                x, y = int(move / 3), int(move % 3)
                self.game_board[x][y] = turn
                value = min(value, self.n_step_rec('X', depth - 1))
                self.game_board[x][y] = '-'

            return value
```

In [1]:
from TicTacToe import TicTacToe
game = TicTacToe(player1="minimax", player2="random")
game.play()
game.percentage()

| -| -| -| 
| -| -| -| 
| -| -| -| 

| -| X| -| 
| -| -| -| 
| -| -| -| 

| -| X| O| 
| -| -| -| 
| -| -| -| 

| -| X| O| 
| -| X| -| 
| -| -| -| 

| -| X| O| 
| O| X| -| 
| -| -| -| 

| -| X| O| 
| O| X| -| 
| -| X| -| 

Player 1 won!
Player 1: minimax vs. Player 2: random
Player 1 W: 78, L: 8, T: 14 

The results are quite similar to those from the Monte Carlo Search Tree. TicTacToe is a ---. And Monte CArlo Simulations converge to the Minimax solution for such games. A final improvement we can provide for the Minimax algorithm is ***Alpha Beta Pruning***. The process of simulating games can be quite time consumming especially for large game trees. A way we can improve the efficiency is to ignore suboptimal moves when a better move is found. To be more precise, 